# **Discrete Choice Modeling for Subsidy Program Implementation**
### *Supplementary Materials & Analytical Framework*

This Jupyter notebook provides the complete analytical framework and code implementation for the paper, *"Graph-Based Geospatial Decision Support System for Optimizing Educational Subsidies."* It details the data preprocessing, modeling, and policy simulation steps used to evaluate the effectiveness of the Educational Service Contracting (ESC) program.

The analysis leverages a pre-constructed **iGraph network** that integrates data from over 47,000 educational institutions, administrative boundaries, road networks, and student mobility patterns.

### **Core Research Question**

The central policy question explored in this analysis is:

> *Given fixed capacity at both public and private junior high schools, how do changes in the ESC subsidy amount, the number of available ESC slots, and total budget affect the distribution of surplus learners across schools, and what is the maximum system-wide surplus reduction achievable?*

### **Notebook Workflow**

This notebook follows a systematic, multi-stage workflow, from data ingestion to policy simulation. Each stage is implemented in the sections below to ensure full reproducibility of our findings.

**1. Setup and Data Loading**
   - **Load Libraries:** Import necessary Python libraries for data analysis, network science, and statistical modeling.
   - **Load Graph Network:** Load the pre-constructed `iGraph` network containing integrated educational and spatial data.

**2. Graph Attribute Enhancement**
   - **Add Grade 6 Enrollment:** Integrate Grade 6 enrollment data as node attributes for origin schools.
   - **Incorporate Financial Data:**
     - Set tuition and ESC subsidy amounts for public schools to zero.
     - Calculate and add `topup_fees` (out-of-pocket costs) as a node attribute.
   - **Add ESC Slot Allocation:** Integrate the number of available ESC subsidy slots for each participating private school.

**3. Data Preprocessing for Choice Modeling**
   - **Graph to Dataframe Conversion:** Convert the `iGraph` network structure into an origin-destination (OD) dataframe.
   - **Spatial Calculation:** Compute a distance matrix between all origin and destination schools.
   - **Choice Set Simulation:** Simulate a binary (public/private) choice set to facilitate discrete choice modeling, as the original data only contains flows to private schools.
   - **Feature Scaling and Aggregation:** Scale monetary and distance variables and aggregate the data to the origin-school level for analysis.

**4. Discrete Choice Estimation**
   - **Model Specification:** Define a binomial logit model to estimate the probability of choosing a private school.
   - **Parameter Estimation:** Fit the model using the preprocessed data to uncover the determinants of school choice at the catchment level.

**5. Policy Scenario and Impact Analysis**
   - **Baseline vs. Policy Simulation:** Simulate a policy change (e.g., a ₱1,000 increase in the ESC subsidy) and predict the resulting shift in student enrollment.
   - **Congestion Analysis:**
     - Evaluate the impact on student congestion in public Junior High Schools (JHS).
     - Assess the resulting demand and capacity utilization changes in private ESC-participating schools.
   - **System-Wide Impact:** Quantify the overall reduction in educational system congestion under the new policy scenario.

# **1. Setup**
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import igraph as ig
import re
import networkx as nx
from typing import Dict, List, Tuple, Optional, Union
from scipy.spatial.distance import cdist
from statsmodels.formula.api import glm
from statsmodels.genmod.families import Binomial
from tqdm import tqdm
from pathlib import Path
import sys

import warnings
warnings.filterwarnings('ignore')

In [2]:
# SETUP: This notebook should be run from the project_paaral_2 directory
# Expected structure:
# project_paaral_2/
#   ├── output/
#   │   └── gnet_flow_Region4A_07-23-2025.pkl
#   └── [this notebook]

PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name != 'project_paaral_2':
    # Try to find project_paaral_2 in current path
    for parent in PROJECT_ROOT.parents:
        if parent.name == 'project_paaral_2':
            PROJECT_ROOT = parent
            break
    else:
        print("⚠ Warning: Not running from project_paaral_2 directory")
        print(f"Current directory: {PROJECT_ROOT}")
        print("Please navigate to the project_paaral_2 directory or update PROJECT_ROOT")

print(f"Using project root: {PROJECT_ROOT}")

# Load data
data_path = PROJECT_ROOT / 'output' / 'gnet_flow_Region4A_07-23-2025.pkl'

try:
    with open(data_path, 'rb') as f:
        data = pickle.load(f)
    print(f"✓ Data loaded successfully from: {data_path}")
except FileNotFoundError:
    print(f"✗ Data file not found at: {data_path}")
    print("Please ensure the file exists in the output directory")

Using project root: /Users/paulamartinez/Documents/CAIR/E-CAIR/paaral/project_paaral_2
✓ Data loaded successfully from: /Users/paulamartinez/Documents/CAIR/E-CAIR/paaral/project_paaral_2/output/gnet_flow_Region4A_07-23-2025.pkl


In [3]:
# Check network
beneficiary_graph = data['beneficiary_graph']
print("Number of nodes:", len(beneficiary_graph.vs))
print("Number of edges:", len(beneficiary_graph.es))

Number of nodes: 6332
Number of edges: 3708


In [4]:
# Node attributes
for node in beneficiary_graph.vs:
    print('Node attributes:', node.attributes(), end='\n\n')
    # print('School attributes of the node:', node['school_attrs'][0].items(),
    #       end='\n\n')
    print('Length of school attributes:', len(node['school_attrs'][0].keys()))
    break

Node attributes: {'school_id': 101583, 'school_type': 'public', 'school_name': 'Dorongan ES', 'x': 560327.2979985776, 'y': 1600265.3025909872, 'school_attrs': [{'region': 'Region I', 'division': 'Pangasinan I, Lingayen', 'school_name': 'Dorongan ES', 'province': 'PANGASINAN', 'municipality': 'MANGATAREM', 'longitude': 121.5609412, 'latitude': 14.4687248, 'shifting_schedule': 'No Shift', 'modified coc': 'Purely ES', 'seats_es': 67.0, 'seats_jhs': nan, 'seats_shs': nan, 'enrollment_es': 181.0, 'enrollment_jhs': nan, 'enrollment_shs': nan, 'shs_ABM': nan, 'shs_ARTS & DESIGN': nan, 'shs_GAS': nan, 'shs_HUMSS': nan, 'shs_PBM': nan, 'shs_SPORTS': nan, 'shs_STEM': nan, 'shs_TVL': nan, 'sector': 'Public', 'x': 560327.2979985776, 'y': 1600265.3025909872}], 'is_school': True}

Length of school attributes: 26


# **2. Add node attributes**
---

## 2.1. Add Grade 6 enrollment

### 2.1.1. Load module

In [5]:
# Setup project path (same as before)
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name != 'project_paaral_2':
    for parent in PROJECT_ROOT.parents:
        if parent.name == 'project_paaral_2':
            PROJECT_ROOT = parent
            break
    else:
        print("⚠ Warning: Not running from project_paaral_2 directory")
        print(f"Current directory: {PROJECT_ROOT}")

print(f"Using project root: {PROJECT_ROOT}")

# Add to Python path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Import and initialize with project root
from modules.datasets import KnowledgeBase

# Change to project root before initializing KnowledgeBase
original_cwd = os.getcwd()
os.chdir(PROJECT_ROOT)

try:
    kb = KnowledgeBase(load_knowledge_base=True)
    print("✓ KnowledgeBase loaded successfully")
finally:
    # Change back to original directory
    os.chdir(original_cwd)

Using project root: /Users/paulamartinez/Documents/CAIR/E-CAIR/paaral/project_paaral_2
Loading public school coordinates as of SY 2023-2024.
Time elapsed for public school coordinates: 2.70 seconds

Loading private school coordinates as of 2024.
Time elapsed for private school coordinates: 2.02 seconds

Loading public and private school enrollment & SHS offerings for SY 2023-2024.
Time elapsed for enrollment & SHS offerings: 4.46 seconds

Loading public & private school furnitures, namely seats, for SY 2023-2024.
Time elapsed for public & private seats: 5.50 seconds

Loading public school shifting schedule for SY 2023-2024.
Time elapsed for public shifting: 20.81 seconds

Loading private school ESC and SHS VP delivering schools as of 2024.
Time elapsed for public shifting: 0.72 seconds

✓ KnowledgeBase loaded successfully


### 2.1.2. Get the Grade 6 enrollment of origin schools

In [6]:
raw_enrollment = kb.enrollment.copy()
pivot_enr = raw_enrollment.pivot_table(
    index='school_id',
    columns='grade_level',
    values='count_enrollment',
    aggfunc='sum'
)
g6_enr = pivot_enr[['Grade 6']] #.reset_index()
g6_enr.columns = ['enr_grade_6']
g6_enr = g6_enr.reset_index()
g6_enr['school_id'] = g6_enr['school_id'].astype(int)

In [7]:
display(g6_enr.head(3))

,school_id,enr_grade_6
0,100000,39.0
1,100001,11.0
2,100002,69.0


### 2.1.3. Add Grade 6 enrollment as node attribute

In [8]:
# Create mapping dictionary
g6_dict = dict(zip(g6_enr['school_id'], g6_enr['enr_grade_6']))

# Update each node's school_attrs dict
for node in beneficiary_graph.vs:
    school_id = node['school_id']

    # Get the G6 enrollment (default to NaN if not found)
    g6_enrollment = g6_dict.get(school_id, float('nan'))

    # Add to the school_attrs dict
    node['school_attrs'][0]['enrollment_g6'] = g6_enrollment

In [9]:
# Verify
for node in beneficiary_graph.vs:
    node_attrs = node.attribute_names()
    school_attrs = node['school_attrs'][0].keys()
    print("Node attributes:", node_attrs, end='\n\n')
    print("School attributes:", school_attrs, end='\n\n')
    print("Length of school attributes", len(school_attrs))
    break

Node attributes: ['school_id', 'school_type', 'school_name', 'x', 'y', 'school_attrs', 'is_school']

School attributes: dict_keys(['region', 'division', 'school_name', 'province', 'municipality', 'longitude', 'latitude', 'shifting_schedule', 'modified coc', 'seats_es', 'seats_jhs', 'seats_shs', 'enrollment_es', 'enrollment_jhs', 'enrollment_shs', 'shs_ABM', 'shs_ARTS & DESIGN', 'shs_GAS', 'shs_HUMSS', 'shs_PBM', 'shs_SPORTS', 'shs_STEM', 'shs_TVL', 'sector', 'x', 'y', 'enrollment_g6'])

Length of school attributes 27


## 2.2. Set tuition fees and subsidy amount of `Public`-sector schools to 0.

The underlying assumption is that public schools have no tuition and other fees.

In [10]:
for node in beneficiary_graph.vs:
    if node['school_attrs'][0]['sector'] == 'Public':
        node['school_attrs'][0]['esc_(total)'] = 0
        node['school_attrs'][0]['esc_amount'] = 0

In [11]:
# Verify:
set([node['school_attrs'][0]['esc_(total)'] for node in beneficiary_graph.vs if node['school_attrs'][0]['sector'] == 'Public'])

{0}

## 2.3. Add top-up fees as node attributes

Note: There NaN values because of private schools without tuition fee information under `esc_(total)`.

In [12]:
for node in beneficiary_graph.vs:
    school_attrs = node['school_attrs'][0]
    node['school_attrs'][0]['topup_fees'] = abs(school_attrs.get('esc_amount', np.nan) - school_attrs.get('esc_(total)', np.nan))

In [13]:
# Verify
set([node['school_attrs'][0].get('topup_fees') for node in beneficiary_graph.vs if node['school_attrs'][0].get('topup_fees') != np.nan])

{0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 49224.82,
 8265.0,
 nan,
 41038.0,
 nan,
 nan,
 16481.25,
 100.0,
 102.5,
 nan,
 24688.0,
 16500.0,
 24695.0,
 nan,
 nan,
 32911.9,
 41108.0,
 nan,
 16550.0,
 24745.0,
 nan,
 nan,
 73930.0,
 nan,
 32983.87,
 16600.0,
 nan,
 24816.9,
 nan,
 24823.940000000002,
 nan,
 33030.0,
 16650.0,
 nan,
 73999.0,
 8486.0,
 nan,
 8500.0,
 nan,
 24886.0,
 16700.0,
 nan,
 nan,
 8512.96,
 24900.0,
 nan,
 nan,
 8537.5,
 nan,
 8545.0,
 41330.0,
 nan,
 33140.0,
 nan,
 nan,
 400.2600000000002,
 24977.0,
 16788.0,
 24983.0,
 nan,
 nan,
 16800.0,
 nan,
 16827.5,
 nan,
 49600.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 500.0,
 33275.0,
 16895.0,
 nan,
 16900.0,
 520.0,
 nan,
 nan,
 nan,
 33300.0,
 nan,
 8729.0,
 49694.15,
 57890.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 16957.24,
 nan,
 57936.0,
 57950.0

## 2.4. Add subsidy slots as node attribute

In [14]:
from modules.dce import load_esc_slots_data_from_excel

In [15]:
df_esc_slots = load_esc_slots_data_from_excel(PROJECT_ROOT)
fixed_slot_map = dict(zip(df_esc_slots['esc_school_id'].astype(int), 
                          df_esc_slots['slot_type_fixed']))
addon_slot_map = dict(zip(df_esc_slots['esc_school_id'].astype(int), 
                          df_esc_slots['slot_type_addon']))
incentive_slot_map = dict(zip(df_esc_slots['esc_school_id'].astype(int), 
                          df_esc_slots['slot_type_incentive']))

for node in beneficiary_graph.vs:
    node_esc_id = node['school_attrs'][0].get('esc_school_id')

    # Add fixed slots
    num_fixed_slots = fixed_slot_map.get(node_esc_id)
    node['school_attrs'][0]['slot_type_fixed'] = num_fixed_slots

    # Add add-on slots
    num_addon_slots = addon_slot_map.get(node_esc_id)
    node['school_attrs'][0]['slot_type_addon'] = num_addon_slots

    # Add aincentive slots
    num_incentive_slots = incentive_slot_map.get(node_esc_id)
    node['school_attrs'][0]['slot_type_incentive'] = num_incentive_slots

Looking for 'Alphalist' files in: /Users/paulamartinez/Documents/CAIR/E-CAIR/paaral/project_paaral_2/datasets/private
Found 3 Alphalist files: ['Alphalist-Schools-Slots-incentive_slots.csv', 'Alphalist-Schools-Slots-fixed_slots.csv', 'Alphalist-Schools-Slots-addon_slots.csv']
  Loading: Alphalist-Schools-Slots-incentive_slots.csv
  Loading: Alphalist-Schools-Slots-fixed_slots.csv
  Loading: Alphalist-Schools-Slots-addon_slots.csv
✓ ESC slots data loaded successfully


# **3. Data Preprocessing**
---

## 3.1. Convert graph to origin-destination dataframe

In [16]:
from modules.dce import extract_flows

In [17]:
df_esc_flows = extract_flows(beneficiary_graph)
df_esc_flows = df_esc_flows[
    (df_esc_flows['destination_sector'] == 'Public') |
    ((df_esc_flows['destination_sector'] == 'Private') & (df_esc_flows['esc_participating'] == 1))
]
print('Shape:', df_esc_flows.shape)
display(df_esc_flows.head(3))

Shape: (2113, 26)


,origin_school_id,destination_school_id,origin_vertex_index,destination_vertex_index,esc_beneficiaries,total_beneficiaries,origin_school_name,destination_school_name,origin_sector,destination_sector,...,esc_fees,seats_jhs_dest,seats_jhs_origin,enrollment_jhs_dest,enrollment_jhs_origin,enrollment_g6_origin,topup_fees,slot_type_fixed,slot_type_addon,slot_type_incentive
0,107247,423589,60,5318,1,1,Sampaga Elementary School,"Core Science Academy, Inc.",Public,Private,...,26200.00,50.0,NaN,258.0,NaN,82.0,17200.00,70.0,0.0,0.0
1,107405,401595,206,3810,2,2,Doña Matilde Memorial Elementray School,"Saint Mary's Educational Institute, Inc.",Public,Private,...,33256.35,225.0,NaN,225.0,NaN,112.0,24256.35,63.0,0.0,0.0
2,107406,401595,207,3810,2,2,Esteban E. Vito Memorial Elementary School,"Saint Mary's Educational Institute, Inc.",Public,Private,...,33256.35,225.0,NaN,225.0,NaN,53.0,24256.35,63.0,0.0,0.0


## 3.2. Calculate distance matrix between origin and destinations

In [18]:
from modules.dce import calculate_distance_matrix

In [19]:
df_esc_flows = calculate_distance_matrix(df_flows=df_esc_flows)
print('Shape:', df_esc_flows.shape)
display(df_esc_flows.head(3))

Calculating distances for 2113 flow records...
✓ Distance calculation complete
✓ Distance range: 13.8m to 210744.4m
✓ Mean distance: 11573.8m
Shape: (2113, 27)


,origin_school_id,destination_school_id,origin_vertex_index,destination_vertex_index,esc_beneficiaries,total_beneficiaries,origin_school_name,destination_school_name,origin_sector,destination_sector,...,seats_jhs_dest,seats_jhs_origin,enrollment_jhs_dest,enrollment_jhs_origin,enrollment_g6_origin,topup_fees,slot_type_fixed,slot_type_addon,slot_type_incentive,distance_meters
0,107247,423589,60,5318,1,1,Sampaga Elementary School,"Core Science Academy, Inc.",Public,Private,...,50.0,NaN,258.0,NaN,82.0,17200.00,70.0,0.0,0.0,3994.301628
1,107405,401595,206,3810,2,2,Doña Matilde Memorial Elementray School,"Saint Mary's Educational Institute, Inc.",Public,Private,...,225.0,NaN,225.0,NaN,112.0,24256.35,63.0,0.0,0.0,3973.318830
2,107406,401595,207,3810,2,2,Esteban E. Vito Memorial Elementary School,"Saint Mary's Educational Institute, Inc.",Public,Private,...,225.0,NaN,225.0,NaN,53.0,24256.35,63.0,0.0,0.0,4220.170371


## 3.3. Simulate public and private OD

#### **Methodological Note: Simulating a Binary Choice Set**

A key challenge in the available administrative data is the absence of observed public elementary-to-public JHS student flows. The `df_esc_flows` dataframe only captures the "revealed preference" for students who chose to enroll in a *private* ESC-participating school.

To build a discrete choice model that estimates the trade-offs between public and private options, we must first construct a choice set that includes both alternatives. We address this by **simulating the public school choice**.

As shown in the cell below, we randomly designate 50% of the unique destination schools as "Public" for modeling purposes. This allows us to create the `chose_private` binary variable and estimate the influence of factors like `esc_amount_k` and `topup_fees_k` on the probability of choosing a private institution.

> **Important:** The following model is therefore a **behavioral model calibrated on observed private school attributes against a synthetically generated set of public alternatives.** Its purpose is to enable policy simulation by quantifying the relative importance of key variables. The model's performance metrics should be interpreted in the context of this simulated environment.

In [20]:
from modules.dce import simulate_public_private_od

In [21]:
df_sim = simulate_public_private_od(df=df_esc_flows, public_ratio=0.5, seed=42)
print('Number of simulated public destination schools:', 
      df_sim['destination_sector'].value_counts()[0])
print('Number of simulated private ESC-participating destination schools:', 
      df_sim['destination_sector'].value_counts()[1])

Converting 76 out of 153 schools to public (50.0%)
Number of simulated public destination schools: 1075
Number of simulated private ESC-participating destination schools: 1038


## 3.4. Scale variables for modeling

In [22]:
df_dce = df_sim.copy()

# Dependent variable
df_dce['chose_private'] = (df_sim['destination_sector'] == 'Private').astype(int)

# Prepare predictors
# Scale variables
df_dce['esc_amount_k'] = df_dce['esc_amount'] / 1_000
df_dce['topup_fees_k'] = df_dce['topup_fees'] / 1_000
df_dce['distance_km'] = df_dce['distance_meters'] / 1_000

print('Shape:', df_dce.shape)
display(df_dce.head(3))

Shape: (2113, 31)


,origin_school_id,destination_school_id,origin_vertex_index,destination_vertex_index,esc_beneficiaries,total_beneficiaries,origin_school_name,destination_school_name,origin_sector,destination_sector,...,enrollment_g6_origin,topup_fees,slot_type_fixed,slot_type_addon,slot_type_incentive,distance_meters,chose_private,esc_amount_k,topup_fees_k,distance_km
0,107247,423589,60,5318,1,1,Sampaga Elementary School,"Core Science Academy, Inc.",Public,Public,...,82.0,17200.00,70.0,0.0,0.0,3994.301628,0,0.0,17.20000,3.994302
1,107405,401595,206,3810,2,2,Doña Matilde Memorial Elementray School,"Saint Mary's Educational Institute, Inc.",Public,Private,...,112.0,24256.35,63.0,0.0,0.0,3973.318830,1,9.0,24.25635,3.973319
2,107406,401595,207,3810,2,2,Esteban E. Vito Memorial Elementary School,"Saint Mary's Educational Institute, Inc.",Public,Private,...,53.0,24256.35,63.0,0.0,0.0,4220.170371,1,9.0,24.25635,4.220170


## 3.5. Aggregate data to origin school-level

In [23]:
df_agg = df_dce.groupby('origin_school_id').agg({
    # Flow data
    'esc_beneficiaries': 'sum', # Total students flowing to private ESC schools

    # Policy variables
    'chose_private': 'mean', # Proportion of flows choosing private ESC from this origin school
    'esc_amount_k': 'mean', # Average ESC amount available to this origin_school
    'distance_km': 'mean', # Average distance of destination schools from this origin school
    'topup_fees_k': 'mean', # Average remaining cost of private tuition fees that a student needs to shoulder

    # School characteristics (same for each origin school)
    'seats_jhs_origin': 'first', # Number of seats at the origin school
    'enrollment_jhs_origin': 'first',
    'enrollment_g6_origin': 'first'
}).reset_index()

print('Shape:', df_agg.shape)
display(df_agg.head(3))

Shape: (1536, 9)


,origin_school_id,esc_beneficiaries,chose_private,esc_amount_k,distance_km,topup_fees_k,seats_jhs_origin,enrollment_jhs_origin,enrollment_g6_origin
0,104712,1,0.0,0.0,1.697110,0.500000,NaN,NaN,19.0
1,104714,7,0.0,0.0,40.297328,23.266125,NaN,NaN,107.0
2,104722,20,0.0,0.0,1.101640,0.500000,NaN,NaN,71.0


# **4. Discrete choice modeling at the origin school-level**
---

### **Reframing the DCE Model for School-Level Data**

Given that our available data is at the **school level**, we need to reconceptualize the discrete choice experiment (DCE) model. Instead of modeling individual student choices, we focus on **aggregate patterns** across geographic units.

- **Unit of Analysis**: School catchment areas or municipalities  
- **Choice Variable**: Aggregate preference for **private vs. public schools** within each area  
- **Interpretation Goal**: Identify the **community- or area-level factors** that increase the likelihood of students enrolling in **ESC-subsidized private schools**

We model the probability that a student from a given origin school attends a private ESC-participating school as a function of school and municipal-level variables:

$$
P(\text{Student from an origin school chooses a private ESC school}) = f\left(
\text{subsidy},\;
\text{topup\_fees},\;
\text{distance}
\right)
$$

This approach allows us to explore how financial incentives, household income levels, and geographic accessibility influence **community-level participation** in the ESC program.


In [24]:
formula = """chose_private ~ 
             esc_amount_k + 
             topup_fees_k + 
             distance_km"""

model = glm(formula, data=df_agg, family=Binomial())
results = model.fit()

In [25]:
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:          chose_private   No. Observations:                 1536
Model:                            GLM   Df Residuals:                     1532
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -119.74
Date:                Tue, 05 Aug 2025   Deviance:                       60.821
Time:                        18:20:19   Pearson chi2:                     77.6
No. Iterations:                     8   Pseudo R-squ. (CS):             0.6894
Covariance Type:            nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.5281      0.423    -10.700   

### **Discussion of Model Results and Limitations**

#### **Interpreting the Distance Coefficient**

The model output shows that the `distance_km` variable is not statistically significant (p=0.623). While counterintuitive, this suggests that at the aggregated catchment-area level, financial factors like the **subsidy amount (`esc_amount_k`)** and **out-of-pocket costs (`topup_fees_k`)** are the dominant drivers of choice. Once a school community has access to a subsidized private option, the average distance to that option appears to be a secondary concern.

#### **Ecological Inference and Scope of Interpretation**

It is critical to note that this model is estimated at the **origin school level**, not the individual student level. Therefore, the results are subject to potential ecological inference limitations. The model is powerful for simulating how policy levers affect school catchments and regions, but the coefficients should not be interpreted as the precise decision-making calculus of individual families.

In [26]:
print(f'McFadden R-squared: {results.pseudo_rsquared(kind='mcf'):.6f}')
print(f'Cox-Snell R-squared: {results.pseudo_rsquared(kind='cs'):.6f}')

McFadden R-squared: 0.882345
Cox-Snell R-squared: 0.689389


# **5. Policy scenario analysis**

In [27]:
from modules.dce import predict_choice_probability, calculate_scenario_metrics

In [28]:
df_scenario = df_agg.copy()

# Current scenario (baseline)
current_data = df_scenario.copy()
current_data['scenario'] = 'Current Policy'

# Proposed scenario: +₱1k increase in ESC amount for all areas
proposed_data = df_scenario.copy()
proposed_data['esc_amount_k'] = proposed_data['esc_amount_k'] + 1.0 # +₱1k
proposed_data['scenario'] = 'Proposed (+₱1k ESC)'

# Calculate probabilities for both scenarios
for df in[current_data, proposed_data]:
    df['predicted_prob'] = predict_choice_probability(
        results=results,
        esc_amount_k=df['esc_amount_k'],
        topup_fees_k=df['topup_fees_k'],
        distance_km=df['distance_km'],
        # is_medium_income_area=df['medium_income_area'],
        # is_high_income_area=df['high_income_area'],
    )

# Aggregate results
current_metrics = calculate_scenario_metrics(data=current_data, 
                                             scenario_name='Current Policy')
proposed_metrics = calculate_scenario_metrics(data=proposed_data,
                                              scenario_name='Proposed (+₱1k)')

In [29]:
df_comparison = pd.DataFrame([current_metrics, proposed_metrics])

# Calculate incremental impact
current_students = float(current_metrics['Total_students_served'].replace(',', ''))
proposed_students = float(proposed_metrics['Total_students_served'].replace(',', ''))
additional_students = proposed_students - current_students

current_cost = float(current_metrics['Total_program_cost'].replace('₱', '').replace(',', ''))
proposed_cost = float(proposed_metrics['Total_program_cost'].replace('₱', '').replace(',', ''))
additional_cost = proposed_cost - current_cost

print(f"\nINCREMENTAL IMPACT OF +₱1k ESC:")
print("=" * 35)
print(f"Additional students served: {additional_students:,.0f}")
print(f"Additional program cost: ₱{additional_cost:,.0f}")
print(f"Cost per additional student: ₱{additional_cost / additional_students if additional_students > 0 else 0:,.0f}")

# System-wide impact calculation
print(f"\nSYSTEM-WIDE IMPACT:")
print("=" * 20)

# Calculate system capacity changes (using your existing analysis)
# This connects to your public/private utilization analysis
total_potential = current_data['enrollment_g6_origin'].sum()
students_switching = additional_students

print(f"Total Grade 6 students: {total_potential:,.0f}")
print(f"Students switching to private: {students_switching:,.0f}")
print(f"System impact: {students_switching/total_potential:.1%} of students switch to private")


INCREMENTAL IMPACT OF +₱1k ESC:
Additional students served: 10,065
Additional program cost: ₱165,775,808
Cost per additional student: ₱16,471

SYSTEM-WIDE IMPACT:
Total Grade 6 students: 217,021
Students switching to private: 10,065
System impact: 4.6% of students switch to private


## **5.1. Impact on public JHS congestion**

In [31]:
##### PUBLIC JHS UTILIZATION ANALYSIS

# What is the new congestion of public JHS destination schools after the policy change?
current_prob = predict_choice_probability(
    results=results,
    esc_amount_k=current_data['esc_amount_k'],
    topup_fees_k=current_data['topup_fees_k'],
    distance_km=current_data['distance_km'],
)
proposed_prob = predict_choice_probability(
    results=results,
    esc_amount_k=proposed_data['esc_amount_k'],
    topup_fees_k=proposed_data['topup_fees_k'],
    distance_km=proposed_data['distance_km'],
)

# Calculate how many students leave each origin school for private schools
df_agg['prob_change'] = proposed_prob - current_prob

# 1. Calculate current students going to public JHS from each origin
df_agg['current_students_to_public_jhs'] = (
    df_agg['enrollment_g6_origin'] - df_agg['esc_beneficiaries']
)

# 2. Calculate how many additional students will choose private (which reduces public JHS enrollment)
df_agg['additional_private_students'] = (
    df_agg['prob_change'] * df_agg['enrollment_g6_origin']
)

# 3. Calculate new students going to public JHS after policy change
df_agg['new_students_to_public_jhs'] = (
    df_agg['current_students_to_public_jhs'] - df_agg['additional_private_students']
)

# 4. Calculate change in public JHS demand
df_agg['change_in_public_jhs_demand'] = (
    df_agg['new_students_to_public_jhs'] - df_agg['current_students_to_public_jhs']
)

print("PUBLIC JHS DEMAND ANALYSIS:")
print("=" * 35)
print(f"Current students going to public JHS: {df_agg['current_students_to_public_jhs'].sum():,.0f}")
print(f"Additional students choosing private: {df_agg['additional_private_students'].sum():,.0f}")
print(f"New students going to public JHS: {df_agg['new_students_to_public_jhs'].sum():,.0f}")
print(f"Reduction in public JHS demand: {abs(df_agg['change_in_public_jhs_demand'].sum()):,.0f}")

# 5. Get unique public JHS destination schools and their current capacities
unique_public_jhs = df_dce[df_dce['destination_sector'] == 'Public'].drop_duplicates('destination_school_id')
df_public_jhs = unique_public_jhs[['destination_school_id',
                                   'enrollment_jhs_dest',
                                   'seats_jhs_dest']].copy()

# Calculate current utilization
df_public_jhs['current_utilization'] = (
    df_public_jhs['enrollment_jhs_dest'] / df_public_jhs['seats_jhs_dest']
)

# 6. Calculate system-wide impact on public JHS schools
total_public_jhs_seats = df_public_jhs['seats_jhs_dest'].sum()
current_public_jhs_enrollment = df_public_jhs['enrollment_jhs_dest'].sum()
students_leaving_public_jhs = df_agg['additional_private_students'].sum()
new_public_jhs_enrollment = current_public_jhs_enrollment - students_leaving_public_jhs

# System-wide utilization
current_public_system_utilization = current_public_jhs_enrollment / total_public_jhs_seats
new_public_system_utilization = new_public_jhs_enrollment / total_public_jhs_seats
public_system_utilization_change = new_public_system_utilization - current_public_system_utilization

print(f"\nSYSTEM-WIDE PUBLIC JHS IMPACT:")
print("=" * 35)
print(f"Number of public JHS schools: {len(df_public_jhs)}")
print(f"Total seats capacity: {total_public_jhs_seats:,.0f}")
print(f"Current enrollment: {current_public_jhs_enrollment:,.0f}")
print(f"Students leaving for private: {students_leaving_public_jhs:,.0f}")
print(f"New enrollment: {new_public_jhs_enrollment:,.0f}")
print()
print(f"Current system utilization: {current_public_system_utilization:.1%}")
print(f"New system utilization: {new_public_system_utilization:.1%}")
print(f"Utilization change: {public_system_utilization_change:+.1%} ({abs(public_system_utilization_change / current_public_system_utilization):.1%} change from the current)")

PUBLIC JHS DEMAND ANALYSIS:
Current students going to public JHS: 210,573
Additional students choosing private: 10,065
New students going to public JHS: 200,508
Reduction in public JHS demand: 10,065

SYSTEM-WIDE PUBLIC JHS IMPACT:
Number of public JHS schools: 76
Total seats capacity: 22,735
Current enrollment: 23,470
Students leaving for private: 10,065
New enrollment: 13,405

Current system utilization: 103.2%
New system utilization: 59.0%
Utilization change: -44.3% (42.9% change from the current)


## **5.2. Impact on private ESC congestion**

In [33]:
##### PRIVATE JHS UTILIZATION ANALYSIS

current_prob = predict_choice_probability(
    results=results,
    esc_amount_k=current_data['esc_amount_k'],
    topup_fees_k=current_data['topup_fees_k'],
    distance_km=current_data['distance_km'],
)
proposed_prob = predict_choice_probability(
    results=results,
    esc_amount_k=proposed_data['esc_amount_k'],
    topup_fees_k=proposed_data['topup_fees_k'],
    distance_km=proposed_data['distance_km'],
)

# Calculate how many students leave each origin school for private schools
df_agg['prob_change'] = proposed_prob - current_prob

# 1. Calculate current students going to private JHS from each origin
## This is already given by the column 'esc_beneficiaries'

# 2. Calculate how many additional students will choose private (which increases private JHS enrollment)
df_agg['additional_private_students'] = (
    df_agg['prob_change'] * df_agg['enrollment_g6_origin']
)

# 3. Calculate new students going to private JHS after policy change
df_agg['new_students_to_private_jhs'] = (
    df_agg['esc_beneficiaries'] + df_agg['additional_private_students']
)

# 4. Calculate change in private JHS demand
## This is already given by the column 'additional_private_students'

print("PRIVATE JHS DEMAND ANALYSIS:")
print("=" * 35)
print(f"Current students going to private JHS: {df_agg['esc_beneficiaries'].sum():,.0f}")
print(f"Additional students choosing private: {df_agg['additional_private_students'].sum():,.0f}")
print(f"New students going to private JHS: {df_agg['new_students_to_private_jhs'].sum():,.0f}")

# 5. Get unique private ESC-participating JHS destination schools and their current capacities
unique_private_jhs = (
    df_dce[
        (df_dce['destination_sector'] == 'Private') &
        (df_dce['esc_participating'] == 1)
        ]
    .drop_duplicates(subset='destination_school_id')
)
unique_private_jhs['total_slots_per_private_esc'] = (
    unique_private_jhs['slot_type_fixed'] +
    unique_private_jhs['slot_type_addon'] +
    unique_private_jhs['slot_type_incentive']
)
df_private_jhs = unique_private_jhs[['destination_school_id', 
                                     'enrollment_jhs_dest', 'seats_jhs_dest',
                                     'total_slots_per_private_esc']].copy()

# Calculate current utilization (by slots and seats)
df_private_jhs['current_utilization_by_seats'] = (
    df_private_jhs['enrollment_jhs_dest'] / df_private_jhs['seats_jhs_dest']
)

df_private_jhs['current_utilization_by_slots'] = (
    df_private_jhs['enrollment_jhs_dest'] / df_private_jhs['total_slots_per_private_esc']
)

# 6. Calculate system-wide impact on private JHS schools
total_private_esc_seats = df_private_jhs['seats_jhs_dest'].sum()
total_private_esc_slots = df_private_jhs['total_slots_per_private_esc'].sum()
current_private_esc_enrollment = df_private_jhs['enrollment_jhs_dest'].sum()
students_joining_private_esc = df_agg['additional_private_students'].sum()
new_private_esc_enrollment = current_private_esc_enrollment + students_joining_private_esc

# System-wide utilization of ESC schools
current_private_esc_system_utilization_by_seats = current_private_esc_enrollment / total_private_esc_seats
new_private_esc_system_utilization_by_seats = new_private_esc_enrollment / total_private_esc_seats
private_esc_system_utilization_change_seats = new_private_esc_system_utilization_by_seats - current_private_esc_system_utilization_by_seats

current_private_esc_system_utilization_by_slots = current_private_esc_enrollment / total_private_esc_slots
new_private_esc_system_utilization_by_slots = new_private_esc_enrollment / total_private_esc_slots
private_esc_system_utilization_change_slots = new_private_esc_system_utilization_by_slots - current_private_esc_system_utilization_by_slots

print(f"\nSYSTEM-WIDE PRIVATE ESC IMPACT:")
print("=" * 35)
print(f"Number of private JHS schools: {len(df_private_jhs)}")
print(f"Total seats capacity: {total_private_esc_seats:,.0f}")
print(f"Total ESC slots: {total_private_esc_slots:,.0f}")
print(f"Current enrollment: {current_private_esc_enrollment:,.0f}")
print(f"Students leaving for private: {students_leaving_public_jhs:,.0f}")
print(f"New enrollment: {new_private_esc_enrollment:,.0f}")
print()
print(f"Current system utilization (seats): {current_private_esc_system_utilization_by_seats:.1%}")
print(f"New system utilization (seats): {new_private_esc_system_utilization_by_seats:.1%}")
print(f"Utilization change (seats): {private_esc_system_utilization_change_seats:+.1%} ({abs(private_esc_system_utilization_change_seats / current_private_esc_system_utilization_by_seats) * 100:.1f}% relative change)")
print()
print(f"Current system utilization (slots): {current_private_esc_system_utilization_by_slots:.1%}")
print(f"New system utilization (slots): {new_private_esc_system_utilization_by_slots:.1%}")
print(f"Utilization change (slots): {private_esc_system_utilization_change_slots:+.1%} ({abs(private_esc_system_utilization_change_slots / current_private_esc_system_utilization_by_slots) * 100:.1f}% relative change)")


PRIVATE JHS DEMAND ANALYSIS:
Current students going to private JHS: 6,454
Additional students choosing private: 10,065
New students going to private JHS: 16,513

SYSTEM-WIDE PRIVATE ESC IMPACT:
Number of private JHS schools: 77
Total seats capacity: 24,274
Total ESC slots: 6,457
Current enrollment: 24,376
Students leaving for private: 10,065
New enrollment: 34,441

Current system utilization (seats): 100.4%
New system utilization (seats): 141.9%
Utilization change (seats): +41.5% (41.3% relative change)

Current system utilization (slots): 377.5%
New system utilization (slots): 533.4%
Utilization change (slots): +155.9% (41.3% relative change)


---
# **Discussion of Model Assumptions and Robustness**

The policy simulations presented are derived from our binomial logistic model. Like any model, it is built upon specific assumptions that are important to discuss. The strength of our computational framework is that it makes these assumptions explicit and allows for them to be tested.

**1. Justification for the `public_ratio=0.5` Parameter:**

* **Design Choice:** In our simulation, we set the ratio of simulated public to private schools at 50/50. This choice was deliberate. In the absence of ground-truth data about the true proportion of viable public school options for students, a 50/50 split represents the point of **maximum uncertainty**. It is the most conservative, uninformative prior, establishing a baseline scenario of equal a priori choice probability without biasing the model toward either public or private schools.

* **Framework Capability:** A key strength of our framework is its flexibility. While this analysis proceeds with a 50/50 split, the simulation module is designed to easily accept different ratios. This enables future policy analysis to test scenarios where, for instance, public schools are assumed to be a more or less prevalent option in the choice set.

**2. On the Stochastic Nature of the Simulation:**

* **Reproducibility:** The simulation involves a random assignment of schools to the 'public' choice set. We use a fixed random seed (`seed=42`) to ensure that the results presented in this notebook are fully reproducible. The purpose of this specific analysis is to provide a clear and replicable demonstration of the end-to-end policy evaluation pipeline.

* **Framework Capability for Uncertainty Quantification:** For a formal policy report, this single-run analysis would naturally be expanded. Our framework is explicitly designed to be looped over multiple random seeds. This capability allows for the generation of confidence intervals around the final impact estimates (e.g., "Additional students served"), thereby quantifying the statistical uncertainty introduced by the simulation step. The ability of our system to support such rigorous uncertainty analysis is one of its core contributions.